<h1>Exploiting Genetic Algorithms for Optimisation </h1>
<br>

<h2> The problem</h2>
We need to make cylindrical containers with fixed volume 30 cubic cm so that, given the diameter you can figure out its height, minimizing the building material. 
This mathematically simple minimization problem could be solved with a genetic algorithm driven by the diameter (or radius) that must always be positive. The rest of the problem definition is up to you. 

In [340]:
#!pip install -r requirements.txt

In [341]:
# Install dependencies
import math
import numpy as np
import time
from tqdm import tqdm
import pickle
import random

## Defining the Problem

We need to optimise the building material of Cilinders of Volume = 30 cm<sup>3</sup>

$$
\begin{array}{rcl}
    Volume & = & \pi r^2 h\\
    h & = & \frac{30}{\pi r^2}\\
    r & = & x
\end{array}
\newline
$$

In order to minimise the used material, we need to minimise the surface area. 
Thus we get the following equation, for $r > 0$:
$$
\newline
\begin{array}{rcl}
    Area & = & 2\pi r^2 + 2\pi h\\
    & = & 2 \pi r^2 + \frac{60}{r}\\
\end{array}
$$

Then, our fitness equation is:
$$
\begin{array}{rcl}
fitness & = & \dfrac{1}{Area}\\
\\
fitness & = & \dfrac{1}{2 \pi r^2 + \frac{60}{r}}

\end{array}
$$



In [342]:

#1. Initial Hyperparameters
np.random.seed(2023)

#2. Define height given diameter
def height (d):
    r = d/2
    pi = math.pi
    #volume = pi * (r**2) * h
    h = 30/(pi*(r**2))
    return h

#3. Define Fitness
def fitness (r):
    #r=d/2
    return 1/(2 * (math.pi) * (r**2) + (60/r))
    
#3. Define selection methods
def ranked_fitness (current_generation, max_radius, n):
    ranked_solutions = []
    for el in current_generation:
        ranked_solutions.append([fitness(decode_r(el, max_radius, n)),decode_r(el, max_radius, n)])
        #added a second element to keep track of solutions and not only fitness
    ranked_solutions.sort()
    ranked_solutions.reverse()
    return ranked_solutions

#4. Define selection methods
def random_choice (generation, max_radius, n):
    choice = np.random.randint(0,len(generation)-1)
    return generation[choice]

def ranked_choice (generation, max_radius, n):
    #based on https://stackoverflow.com/questions/20290831/how-to-perform-rank-based-selection-in-a-genetic-algorithm
    generation = generation[:10]
    gen_fit = []
    probabilities = []
    amax = 1.2
    amin = 2 - amax
    m = len(generation)
    
    for i in range(len(generation)-1):
        gen_fit.append([fitness(decode_r(generation[i], max_radius, n)),generation[i]])
        p = (amax - (amax-amin)*(i/(m-1)))*(1/m)
        probabilities.append(p)
        gen_fit.sort(reverse=True)
    #make sure probs sum to 1    
    sum_probs = sum(probabilities)
    probabilities[0] = probabilities[0] + (1 - sum_probs)

    #sorting
    
    choice= gen_fit[np.random.choice(np.arange(len(gen_fit)), p=probabilities)]
    return choice[1]

def wheel_selection (generation, max_radius, n):
    #create list with radius and its fitness
    gen_fit=[]
    for el in generation:
        gen_fit.append([fitness(decode_r(el, max_radius, n)),el])           
    # computes the total the generation fitness
    generation_fitness = sum([el[0] for el in gen_fit])
    # computes the probability for each radius
    probabilities = [el[0]/generation_fitness for el in gen_fit]
    # selects one chromosome based on the computed probabilities
    choice= gen_fit[np.random.choice(np.arange(len(gen_fit)), p=probabilities)]
    return choice[1]


#5. Define Reproduction methods    
def crossover(parentA, parentB):
    cross_point = np.random.randint(len(parentA))
    child = parentA[:cross_point]+parentB[cross_point:]
    return child

#Mutations
def mutation(parent, probability):
    flipped = lambda x: '1' if x == '0' else '0'
    chars = (flipped(sym) if random.random() < probability else sym for sym in parent)
    return ''.join(chars)
    

#6. Reproduction Behaviour
def reproduction (current_gen, max_radius, n, sel_function):
    new_generation = []
    for i in range(len(current_gen)):
        parentA = sel_function(current_gen, max_radius, n)
        parentB = sel_function(current_gen, max_radius, n)
        child = crossover(parentA, parentB)
        #child=mutation(child, 0.1)
        if '1' in child:
            new_generation.append(child)
        else:
            new_generation.append(parentA)
    return new_generation

        
#Function to decode binary radius
def decode_r(r, max_radius, n):
    rad=list(map(int, str(r)))
    dec_r=(max_radius/(2**n-1))*sum(rad[i]*2**i for i in range(len(rad)))
    return dec_r

#Generation zero binary
def first_gen(pop_size,n):
    gen_zero=[]
    for i in range(pop_size):
        x = []
        while 1 not in x:
            x = [np.random.choice([0, 1]) for _ in range(n)]   
        el = ''.join(map(str, x))
        gen_zero.append(el)

    return gen_zero

In [343]:
#5. New generation iterations
def digi_evolve(pop_size, max_generations, max_radius, n, sel_function_tuple):
    #create temporary vars
    sel_function = sel_function_tuple[0]
    sel_function_name = sel_function_tuple[1]
    fittest = []
    all_gens = {
        'radius'        :   [],
        'generation'    :   [],
        'fitness'       :   [], 
        'selection_type':   []     
    }
    curr_generation = first_gen(pop_size, n) #initialising first gen
    #GA begins
    print('Evolutionary period begins')
    for i in tqdm(range(max_generations)):
        #if i % 50 == 0:
        #    print(f'Generation {i} is going into the world')
        #    print(f'Population size {len(curr_generation)}')
        ranked_fit = ranked_fitness(curr_generation, max_radius, n)
        fittest.append([ranked_fit[0], i])
        for element in ranked_fit:
            #print('###################### new var', element)
            all_gens['radius'].append(element[1])
            all_gens['fitness'].append(element[0])
            all_gens['generation'].append(i)
            all_gens['selection_type'].append(sel_function_name)
        #reproduction
        curr_generation = reproduction(curr_generation, max_radius, n, sel_function)
    print('Evolutionary period ended')
    return fittest, all_gens


## Solution

Solving by hand, we arrive at the following results:

$$
\begin{array}{rcl}
fitness & = & \dfrac{1}{2 \pi r^2 + \frac{60}{r}}\\
\dfrac{d(fitness)}{dr} & = & 4 \pi r - \frac{60}{r^2} > 0\\
4 \pi r & > & \dfrac{60}{r^2}\\\\
4 \pi & > & \dfrac{60}{r^3}\\\\
\dfrac{4 \pi}{60} & > & \dfrac{1}{r^3}\\
\end{array}
$$

where we get:

$$
\begin{array}{rcl}
 r^3 & > & \dfrac{60}{4 \pi} = \dfrac{15}{\pi}
\end{array}
$$

finally:

$$
\begin{array}{rcl}
r_{att} & = & \sqrt[3]{\dfrac{15}{\pi}} = 1.6841
\end{array}
$$

In [344]:
#main
pop_size = 100
max_generations = 500
n=8 #step:0.1176 if we work in range [0,30]
max_radius=30
selection_methods = [[random_choice, 'Random Selection'], [ranked_choice, 'Ranked Selection'],[wheel_selection, 'Wheel Selection']]
best_overall = []
all_generations = []


start_total = time.time()
start_local = time.time()

for selectionMethod in selection_methods:
    print(f'Selection method: {selectionMethod[1]}')
    best_of_each_gen, all_gens = digi_evolve(pop_size, max_generations, max_radius, n, selectionMethod)
    print('The best radius is:', max(best_of_each_gen)[0][1], 'in generation: ',max(best_of_each_gen)[1])

    best_overall.append([max(best_of_each_gen), selectionMethod[1]])
    all_generations.append(all_gens)

    end_local = time.time()
    print('Time in seconds to complete iteration of', selectionMethod[1] ,end_local - start_local, '\n')
    start_local = time.time()


end_total = time.time()
print('Time in seconds to complete entire cycle:',end_total - start_total, '\n')

print('The best radius is:', max(best_overall))

#do we want to add mutations too?


Selection method: Random Selection
Evolutionary period begins


 11%|█         | 54/500 [00:00<00:00, 530.88it/s]

100%|██████████| 500/500 [00:00<00:00, 526.10it/s]


Evolutionary period ended
The best radius is: 1.6470588235294117 in generation:  42
Time in seconds to complete iteration of Random Selection 0.9691002368927002 

Selection method: Ranked Selection
Evolutionary period begins


100%|██████████| 500/500 [00:06<00:00, 78.86it/s]


Evolutionary period ended
The best radius is: 1.8823529411764706 in generation:  0
Time in seconds to complete iteration of Ranked Selection 6.354942321777344 

Selection method: Wheel Selection
Evolutionary period begins


100%|██████████| 500/500 [00:38<00:00, 13.13it/s]

Evolutionary period ended
The best radius is: 1.6470588235294117 in generation:  156
Time in seconds to complete iteration of Wheel Selection 38.09217166900635 

Time in seconds to complete entire cycle: 45.41621422767639 

The best radius is: [[[0.01870081198259601, 1.6470588235294117], 156], 'Wheel Selection']


In [345]:
#Save object for Data Analysis
fileObj = open('data_algos.pkl', 'wb')
pickle.dump(all_generations,fileObj)
fileObj.close()

In [346]:
#Testing with different n
bits=range(7,15,1)
pop_size=10
max_generation=10
max_radius=30
results=[]
for k in bits:
    best_of_each_gen = digi_evolve(pop_size, max_generations, max_radius, k, [wheel_selection, 'Wheel Selection'])
    results.append(max(best_of_each_gen[0]))
print(results)

Evolutionary period begins


100%|██████████| 500/500 [00:00<00:00, 646.06it/s]


Evolutionary period ended
Evolutionary period begins


100%|██████████| 500/500 [00:00<00:00, 557.37it/s]


Evolutionary period ended
Evolutionary period begins


...

Evolutionary period ended
Evolutionary period begins


100%|██████████| 500/500 [00:00<00:00, 628.27it/s]


Evolutionary period ended
Evolutionary period begins


100%|██████████| 500/500 [00:00<00:00, 538.70it/s]

Evolutionary period ended
[[[0.01653979427827921, 2.3622047244094486], 6], [[0.007194358768301489, 4.470588235294118], 2], [[0.017192124183791416, 2.23091976516634], 499], [[0.018483851085179057, 1.8768328445747802], 499], [[0.013117987662652673, 2.9897410845139225], 8], [[0.018254750885682448, 1.9633699633699633], 12], [[0.01617273645304967, 2.431937492369674], 7], [[0.01829933932102217, 1.9483611060245376], 15]]
